In [32]:
import pymongo
import json
import requests
import os
from dotenv import load_dotenv


In [33]:
#Loading my Key and database string
load_dotenv()
API_KEY = os.getenv('KEY')
DB_CONNECTION = os.getenv('DB_string')

In [43]:
DB_NAME = "BetMe"

# Establish a connection with mongoDB
client = pymongo.MongoClient(DB_CONNECTION)

# Create a DB
dataBase = client[DB_NAME]

In [45]:
def checkExistence_DB(DB_NAME, client):
    """It verifies the existence of DB"""
    DBlist = client.list_database_names()
    if DB_NAME in DBlist:
        print(f"DB: '{DB_NAME}' exists")
        return True
    print(f"DB: '{DB_NAME}' not yet present OR no collection is present in the DB")
    return False


_ = checkExistence_DB(DB_NAME=DB_NAME, client=client)

DB: 'BetMe' not yet present OR no collection is present in the DB


In [48]:
COLLECTION_NAME = "All_Sports"
collection = dataBase[COLLECTION_NAME]

In [49]:
# let's verify whether we have our database in the list or not 
# we'll use the following function:-

def checkExistence_COL(COLLECTION_NAME, DB_NAME, db):
    """It verifies the existence of collection name in a database"""
    collection_list = db.list_collection_names()
    
    if COLLECTION_NAME in collection_list:
        print(f"Collection:'{COLLECTION_NAME}' in Database:'{DB_NAME}' exists")
        return True
    
    print(f"Collection:'{COLLECTION_NAME}' in Database:'{DB_NAME}' does not exists OR \n\
    no documents are present in the collection")
    return False


_ = checkExistence_COL(COLLECTION_NAME=COLLECTION_NAME, DB_NAME=DB_NAME, db=dataBase)

Collection:'All_Sports' in Database:'BetMe' does not exists OR 
    no documents are present in the collection


In [66]:
#Creating collection for storing all the available sports

Sports = dataBase.create_collection("All_Sports")


In [69]:
#Getting the data from API

def create_all_sports_data():    
    try:
        request_response = requests.get('https://api.the-odds-api.com/v3/sports', params={
        'api_key': API_KEY})
     
        sports_json = json.loads(request_response.text)
        Sports.insert_many(sports_json['data'])   
       
    except Exception as e:
        print(str(e))


create_all_sports_data()

In [ ]:
#Creating collections for all the upcoming games irrespection of region
total_in_play = dataBase.create_collection("in-play_all_regions")


In [74]:
regions = ["uk", "us", "eu", "au"]

def total_upcoming_sports():
    for region in regions:
        sport_key = 'upcoming'
        try:
            odds_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
                'api_key': API_KEY,
                'sport': sport_key,
                'region': regions, 
                'mkt': 'h2h', 
                'dateFormat': 'iso'})
            
            in_play_data = json.loads(odds_response.text)
            total_in_play.insert_many(in_play_data['data'])
            
        except Exception as e:
            print(str(e))
                

total_upcoming_sports()